In [2]:
import numpy as np
import pandas as pd
from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [3]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=1500,
)
timestamps = candles[:, CandleBodyType.Timestamp]
datetimes = timestamps.astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
close_prices = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [61]:
falling_length = 5
rising_length = 5

falling_length -= 1
rising_length -= 1

open_prices = candles[:, CandleBodyType.Open]
prev_open_prices = np.roll(open_prices, 1)
prev_open_prices[0] = np.nan

close_prices = candles[:, CandleBodyType.Close]
prev_close_prices = np.roll(close_prices, 1)
prev_close_prices[0] = np.nan

current_green_candles = open_prices < close_prices
prev_green_candles = prev_open_prices < prev_close_prices

current_red_candles = open_prices > close_prices
prev_red_candles = prev_open_prices > prev_close_prices

close_diff = np.diff(close_prices, prepend=np.nan)

red_candles = close_diff < 0
green_candles = close_diff > 0

falling_bools = np.full_like(close_diff, False, dtype=np.bool_)
rising_bools = np.full_like(close_diff, False, dtype=np.bool_)

for x in range(falling_length, close_diff.size):
    if red_candles[x - falling_length : x + 1].all():
        falling_bools[x] = True

    if green_candles[x - rising_length : x + 1].all():
        rising_bools[x] = True

falling_bools = np.roll(falling_bools, 1)
falling_bools[0] = False
rising_bools = np.roll(rising_bools, 1)
rising_bools[0] = False

In [59]:
bullish_engulfing_candles = (close_prices > prev_open_prices) & (open_prices <= prev_close_prices)

bullish_engulfing_signals = current_green_candles & prev_red_candles & bullish_engulfing_candles & falling_bools

bearish_engulfing_candles = (close_prices < prev_open_prices) & (open_prices >= prev_close_prices)

bearish_engulfing_signals = current_red_candles & prev_green_candles & bearish_engulfing_candles & rising_bools

In [60]:
plot_engulfing_candles(candles=candles, bearish_signals=bearish_engulfing_signals, bullish_signals=bullish_engulfing_signals)